In [ ]:
! pip install pytube

In [ ]:
! pip install youtube-transcript-api

In [ ]:
! pip install SpeechRecognition

In [ ]:
! pip install git+https://github.com/openai/whisper.git soundfile

In [ ]:
! pip install transformers datasets evaluate jiwer

In [58]:
import speech_recognition as sr
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline

In [91]:
import evaluate
wer = evaluate.load("wer")

## 1.
*   [youtube video](https://www.youtube.com/watch?v=6V-AWMZNvBM&t=10s)
*   аудио скачено с помощью библиотеки `pytube` (формат mp4)
*   субтитры скачены с помощью `youtube_transcript_api`
*   файл конвертирован из mp4 в wav через https://convertio.co/ru/mp4-wav/
*   файл обрезан до 01:34 (~11 предложений) через https://mp3cut.net/

In [37]:
# все доступные аудио
link = "https://www.youtube.com/watch?v=6V-AWMZNvBM&list=WL&index=1"
yt = YouTube(link)
yt.streams.filter(only_audio=True)

[<Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">, <Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">, <Stream: itag="249" mime_type="audio/webm" abr="50kbps" acodec="opus" progressive="False" type="audio">, <Stream: itag="250" mime_type="audio/webm" abr="70kbps" acodec="opus" progressive="False" type="audio">, <Stream: itag="251" mime_type="audio/webm" abr="160kbps" acodec="opus" progressive="False" type="audio">]

In [6]:
# выбор аудио и скачивание
stream = yt.streams.get_by_itag(139)
stream.download()

'/content/the only book ive rated 5 stars in 2023 (so far).mp4'

In [39]:
# доступные субтитры (автосгенерированные)
yt.captions

{'a.en': <Caption lang="English (auto-generated)" code="a.en">}

In [74]:
# скачивание субтитров
yt_tr = YouTubeTranscriptApi.get_transcript('6V-AWMZNvBM?si=9H-Lw4H6BaNcKfba')
yt_tr[:3]

[{'text': 'hey guys so welcome back to my second',
  'start': 0.0,
  'duration': 4.92},
 {'text': 'Channel welcome back to Jack In The Box',
  'start': 2.94,
  'duration': 6.02},
 {'text': 'I have to make this video I feel like',
  'start': 4.92,
  'duration': 4.04}]

In [86]:
txt_all = ''
for i in range(len(yt_tr)):
  txt_all += ' ' + yt_tr[i]['text']
txt = txt_all.split("they're loud")[0].strip()

In [87]:
txt

"hey guys so welcome back to my second Channel welcome back to Jack In The Box I have to make this video I feel like that I have to tell as many people as humanly possible about this book because this is the first five star book that I've read this year the first book I have personally rated five stars and I've read like 38 books I want to say so this is my favorite of the year so far this is Foster by Claire Keegan it's actually my second Claire Keegan book of the year and my second favorite book that I read this year was her other one so I need I need to tell you about them so her other novel is small things like these it was shortlisted for the booker Prize last year and that's kind of what brought it to my attention and I realized after reading it and then going to Dublin that I've really been loving Irish books or books written by Irish people small things like these as a title actually kind of sums up my attitude towards Irish fiction in general because I do think that Irish fict

## 2. Распознование аудио с помощью `SpeechRecognition` (OpenAI Whisper, Google Web Speech API) и `HuggingFace transformers`

In [68]:
r = sr.Recognizer()
sound = sr.AudioFile('the-only-book (cut).wav')
with sound as source:
    audio = r.record(source)

In [69]:
whisp = r.recognize_whisper(audio)
whisp

" Hey guys, so welcome back to my second channel, welcome back to Jack in the books. I have to make this video. I feel like it's a public service at this point that I have to tell as many people as humanly possible about this book because this is the first five star book that I've read this year. The first book I have personally rated five stars and I've read like 38 books I want to say so this is my favourite of the year so far. This is Foster by Claire Keegan. It's actually my second Claire Keegan book of the year and my second favourite book that I read this year was her other one so I need to tell you about them. So her other novel is small things like these. It was shortlisted for the book of prize last year and that's kind of what brought it to my attention and I realised after reading it and then going to Dublin that I've really been loving Irish books or books written by Irish people. Small things like these as a title actually kind of sums up my attitude towards Irish fiction 

In [70]:
goog = r.recognize_google(audio)
goog

"hey guys so welcome back to my second Channel welcome back to Jack In The Box I have to make this video I feel like it's a public service at this point but I have to tell as many people as humanly possible about this book because this is the first five star book that I've read this year the first book I have personally rated 5 stars and I've read like 38 books I want to say so this is my favorite of the year so far this is faster by Keegan is actually my second book of the year and my second favorite book that I read this year was her other one so I need I need to tell you about them so how are the novel is small things like these it was shortlisted for the book of Prize last year and that's kind of what brought it to my attention and I realized after reading it and then going to Dublin Irish books or books written by Irish people small things like this as a title actually kind of sums up my attitude towards Irish fiction in general because I do think the Irish fiction really focuses 

In [73]:
transcriber = pipeline("automatic-speech-recognition", "facebook/wav2vec2-base-960h")
transcr = transcriber('the-only-book (cut).wav')
transcr

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed']
You sho

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

{'text': "HAGIZE SO WELCOME BACK TO MY SECOND CHANNEL WELE COME BACK TO JACK IN THE BORKS I HAVE TO MAKE THIS VIDIA I FELL LIKE ITS OF PUBLIC SERVICE AT THIS POINT THAT I HAVE TO TELL AS MANY PEOPLE AS HUMANLY POSSIBLE ABOUT THIS BOOK BECAUSE THIS IS THE FIRST FIVE STAR BOK THAT I'VE READ THIS YEAR THE FIRST BOOK I PARSONALY RATED FIVE STARS I'VE READ LIKE THAT HE A BOK OMEONL SAY SO THIS IS MY FAVOR OF THE YEARS SO FAR THIS IS FOSTER BY CLAKEGON IS ACTUALLY MY SECOND CLAHEGAN BOOK OF THE YEAR AND MY SECOND FAVORITE BOOK THAT I READ THIS YEAR WAS HAR OTHER ONE SIR I CONNEED I NEED TO TELL YOU ABOUT THEM SO HOT OTHER NOVEL IS SMALL THINGS LIKE THESE IT WAS SHORTLESTED FOR THE BOOK APRIZE LAST YEAR AND THAT'S CON OF WHAT BROUGHT ITS MY ATTENTION AND I REALIZE AFTER READING IT AND IN GOING TO DUBLIN THAT I'VE REALLY BEEN LOVING I RISH BOKS OR BOOKS WRITTEN BY IRISH PEOPLE SMALL THINGS LIKE THESE AS A TITLE ACTUALLY CHAVE SUMS UP MY ATTITUDE TOWARDS IRISH FICTION IN GENERAL BECAUSE I DO TH

## 3.  Сравнение систем и автогенерации YouTube

In [89]:
# эталон
golden = """
hey guys so welcome back to my second Channel welcome back to Jack In The Books I have to make this video I feel like it's a public service at this point
but I have to tell as many people as humanly possible about this book because this is the first five star book that I've read this year the first book
I have personally rated 5 stars and I've read like 38 books I want to say so this is my favorite of the year so far this is Foster by Claire Keegan is actually
my second Claire Keegan book of the year and my second favorite book that I read this year was her other one so I need I need to tell you about them so her other novel
is small things like these it was shortlisted for the Booker Prize last year and that's kind of what brought it to my attention and I realized after reading
it and then going to Dublin that i've really been loving Irish books or books written by Irish people small things like these as a title actually kind of
sums up my attitude towards Irish fiction in general because I do think the Irish fiction really focuses on so much of Irish fiction concerns itself with
telling domestic stories that aren't grand they're not about royalty they're not about celebrities Award winners they're about normal people accidental Sally Rooney pun in there
as well for another bit of Irish fiction and although ostensibly they seem to focus on small lives Small Things smaller currencies actually they are so huge and
important and significant and though they seem quiet they are screaming
"""

In [94]:
# HuggingFace transformers
reference = golden.lower()
hypothesis = transcr['text'].lower()
wer.compute(references=[reference], predictions=[hypothesis])

0.2198581560283688

In [96]:
# Google Cloud Speech
reference = golden.lower()
hypothesis = goog.lower()
wer.compute(references=[reference], predictions=[hypothesis])

0.08156028368794327

In [98]:
# OpenAI Whisper library
reference = golden.lower()
hypothesis = whisp.replace(',', '').replace('.', '').lower().strip()
wer.compute(references=[reference], predictions=[hypothesis])

0.06382978723404255

In [100]:
# Youtube
reference = golden.lower()
hypothesis = txt.lower()
wer.compute(references=[reference], predictions=[hypothesis])

0.05319148936170213

У Youtube результат лучше всего:

| ASR Systems             | WER                   |
|:-----------------------:|:---------------------:|
|HuggingFace transformers |0.2198581560283688     |
|Google Cloud Speech      |0.08156028368794327    |
|OpenAI Whisper library   |0.06382978723404255    |
|Автогенерация Youtube    |**0.05319148936170213**|